In [1]:
celeba_dir = './celeba'
result_dir = './celeba_result'

batchSize = 32
nepoch = 20
nz = 100 # number of noise dimension
nc = 3 # number of result channel
lr = 0.0002
betas = (0.0, 0.99) # adam optimizer beta1, beta2

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.utils as vutils
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.autograd import Variable

In [3]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            nn.ConvTranspose2d(nz, 512, 4, 1, 0, bias=False),
            nn.BatchNorm2d(512),
            nn.ReLU(True),
            nn.ConvTranspose2d(512, 256, 4, 2, 1, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            nn.ConvTranspose2d(256, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            nn.ConvTranspose2d(128, 64, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            nn.ConvTranspose2d(64, nc, 4, 2, 1, bias=False),
            nn.Tanh(),
        )
    
    def forward(self, x):
        return self.main(x)

In [4]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            nn.Conv2d(nc, 64, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(128, 256, 4, 2, 1, bias=False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(256, 512, 4, 2, 1, bias=False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(512, 1, 4, 1, 0, bias=False),
        )
    
    def forward(self, x):
        return self.main(x).view(-1, 1)

In [5]:
class Trainer:
    def __init__(self):
        self.generator = Generator()
        self.discriminator = Discriminator()
        self.loss = nn.MSELoss()
        self.optimizer_g = optim.Adam(self.generator.parameters(), lr=lr, betas=betas)
        self.optimizer_d = optim.Adam(self.discriminator.parameters(), lr=lr, betas=betas)
        
        self.generator.cuda()
        self.discriminator.cuda()
        self.loss.cuda()        
        
    def train(self, dataloader):
        noise = Variable(torch.FloatTensor(batchSize, nz, 1, 1).cuda())
        label_real = Variable(torch.FloatTensor(batchSize, 1).fill_(1).cuda())
        label_fake = Variable(torch.FloatTensor(batchSize, 1).fill_(0).cuda())
        for epoch in range(nepoch):
            for i, (data, _) in enumerate(dataloader, 0):
                # train discriminator
                self.discriminator.zero_grad()

                batch_size = data.size(0)
                label_real.data.resize(batch_size, 1).fill_(1)
                label_fake.data.resize(batch_size, 1).fill_(0)
                noise.data.resize_(batch_size, nz, 1, 1).normal_(0, 1)
                
                real = Variable(data.cuda())
                d_real = self.discriminator(real)

                fake = self.generator(noise)
                d_fake = self.discriminator(fake.detach()) # not update generator
                
                d_loss = self.loss(d_real, label_real) + self.loss(d_fake, label_fake) # real label
                d_loss.backward()
                self.optimizer_d.step()

                # train generator
                self.generator.zero_grad()
                d_fake = self.discriminator(fake)
                g_loss = self.loss(d_fake, label_real) # trick the fake into being real
                g_loss.backward()
                self.optimizer_g.step(  )   
            vutils.save_image(fake.data, '{}/result_epoch_{:03d}.png'.format(result_dir, epoch), normalize=True)

In [6]:
dataset = dset.ImageFolder(celeba_dir, transform=transforms.Compose([
    transforms.CenterCrop(178),
    transforms.Resize(64),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
]))
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batchSize, shuffle=True)
                           
trainer = Trainer()
trainer.train(dataloader)

C:\Users\user\Anaconda3\lib\site-packages\torch\tensor.py:255: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
